### Run this in Python kernel

In [1]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

Syntax error compiling at (REPL:0:0).
Unable to resolve symbol: %%html in this context
Syntax error compiling at (REPL:0:0).
Unable to resolve symbol: <style> in this context
Syntax error compiling at (REPL:0:0).
Unable to resolve symbol: body in this context
Syntax error reading source at (REPL:4:17).
Invalid token: font-family:


class clojure.lang.ExceptionInfo: 

### Run below in the backtesting_clojure kernel

In [1]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure.test :refer :all]
            [clojure-backtesting.data :refer :all]
            [clojure-backtesting.data-management :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            ;;[clojure-backtesting.parameters :refer :all]
            [clojure.string :as str]
            [clojure.pprint :as pprint]
            [java-time :as t]
            [clojupyter.kernel.version :as ver]
            [clojure.edn :as edn]
            [clojupyter.misc.helper :as helper]
  ) ;; require all libriaries from core
  (:use clojure.pprint)
)

nil

### Import dataset

In [2]:
; path to dataset = "../resources/CRSP-extract.csv"
; change it to the relative to your own dataset
;
(reset! data-set (add-aprc (read-csv-row "../resources/CRSP-extract.csv")));

### Get list of tickers with ROE

In [2]:
;; WRITE CHECK YEAR CONDITION
;; Amended to include the ROE for each stock using the merged data
;; output a list of ROE as well
;; Use merged database CRSP + COMPUSTAT
;; output {{:tic "AAPL" :year "1980" :ROE x.x}{...}{:ROE 1.x 2.x 3.x 10.x}}


;; change to lazily later with Leo's API
(defn get-set
    "return a set of maps of tickers and datadate" ;;{:tic "AAPL", :datadate "1981/3/31", :ROE x.x}
    [file date]
    (loop [remaining file
            result-set []
            ROE-list [] ]
        (if (empty? remaining)
            ;; how to change the output here?
            ;;(into #{} (conj result-set {:ROE-set ROE-list}))
            {:data result-set :ROE-set ROE-list}
            (let [first-line (first remaining)
                next-remaining (rest remaining)
                ;;next-result-set (conj result-set (get first-line :datadate)
                [year month day] (map parse-int (str/split date #"-"))]
                ;;merged file use :datadate as key
                (if (= date (get first-line :datadate))
                    (let [[niq PRC cshoq] (map edn/read-string [(get first-line :niq)(get first-line :PRC)(get first-line :cshoq)])
                          ROE (/ niq (* PRC cshoq))]
                        (recur next-remaining (conj result-set {:tic (get first-line :tic) :year year :ROE ROE}) (conj ROE-list ROE))
                    )
                    (recur next-remaining result-set ROE-list)
                )  
            )
        )
    )
)

#'clojure-backtesting.demo/get-set

In [3]:
;; this function can be used to get the list of stocks available for trading
(def dataFA (get-set (read-csv-row "../resources/data-testing-merged.csv") "1980-12-18"))
(println dataFA) ;

{:data [{:tic AAPL, :year 1980, :ROE 0.002731506525434953} {:tic GM, :year 1980, :ROE -0.046370846428037134} {:tic IBM, :year 1980, :ROE 0.0238072629827958}], :ROE-set [0.002731506525434953 -0.046370846428037134 0.0238072629827958]}


In [4]:
;; fn to get a set of ROE
(defn get-ROE
            "return a set of ROE"  ;;{10.2 1.8 x.x ...}
        [dataset]
        (into #{} (get dataset :ROE-set))
        )

#'clojure-backtesting.demo/get-ROE

In [10]:
(defn get-roe-20
    "return a set tickers" ;;{"AAPL" "GM"}
    [data roe-20]
    (loop [remaining (get data :data)
            result-set []]
        (if (empty? remaining)
            (into #{} result-set)
            (let [first-line (first remaining)
                next-remaining (rest remaining)]
                ;;conj the stock ticker whose ROE >= roe-20
                ;;(if (not= -1 (compare roe-20 (get first-line :ROE)))
                (if (>= (get first-line :ROE) roe-20)
                    (recur next-remaining (conj result-set (get first-line :tic)))           
                    (recur next-remaining result-set)            
                )  
            )
        )
    )
)

#'clojure-backtesting.demo/get-roe-20

In [11]:
;; test examples

;; dataFA is the dataset that contains the output from get-set

;; get a set of ROE
(def roe-list-demo (get-ROE dataFA))
(println roe-list-demo)


;; get a sorted set of ROE
(def roe-sorted-set (apply sorted-set roe-list-demo))
(println roe-sorted-set)


;; get a sorted vector of ROE
(def roe-sorted-vec (into '[] roe-sorted-set))
(println roe-sorted-vec)

;; find the 20th per centile ROE
(def roe-20-demo (nth roe-sorted-vec (int (* 0.8 (count roe-sorted-vec)))))
(println roe-20-demo)
         
;; get the list of tickers to buy
(def stock-to-buy-test (get-roe-20 dataFA roe-20-demo))
(println stock-to-buy-test)
(println (type stock-to-buy-test)) ;;stored in a set format

#{0.0238072629827958 -0.046370846428037134 0.002731506525434953}
#{-0.046370846428037134 0.002731506525434953 0.0238072629827958}
[-0.046370846428037134 0.002731506525434953 0.0238072629827958]
0.0238072629827958
#{IBM}
clojure.lang.PersistentHashSet


nil

### Initialise portfolio

In [7]:
;; initialise with current date and initial capital (= $1,000,000)
(init-portfolio "1980-12-18" 100000);

In [13]:
(keys (deref available-tics-))

("AAPL" "F" "IBM")

In [9]:
(def start-date (get-date))

;; define the end date
(def end-date "1981-12-16") ; assume end-date later than start-date
  
(while (not= end-date (get-date))
    ;; check whether the start day is in the dataset
    ;; find the quarterly date before inputting into get-set function
    
    (do 
        ;; write your trading strategy here
        
        ;; get stock tickers and ROE data
        ;; output {{:tic "AAPL" :year "1980" :ROE x.x}{...}{:ROE 1.x 2.x 3.x 10.x}}
        (def stock-data (get-set "../resources/data-testing-merged.csv" end-date))
        
        ;;1. sort the stocks according to their ROE (= Net Income/Total Equity)
        ;;Compute the latest quarter ROE by using: niq/(share price * cshoq) in the merged dataset
        
        ;;1.1 get a set of ROE
         
        (def roe-list (get-ROE stock-data))
        
        ;;sorting function, to determine the top 20% stocks to be bought, return the tickers of the top 20% stocks
            ;;use sorted-set built-in fn in clojure to sort the values of the map returned by get-roe to determine what are the top 20% stocks     
            ;; and turn the sorted-set into list format
            (def roe-sorted (into '[] (apply sorted-set roe-list)))
            
            ;;find the 20% cut-off ROE value
            (let roe-20 (nth roe-sorted (int (* 0.8 (count roe-sorted))))
                      
            ;;get the tickers of the top 20 with function get-roe-20
            (def stocks-to-buy (get-roe-20 stock-data roe-20))
                
            )
                
        ;;2. buy the top 20% stocks and the sell the stocks that are not in the top 20% this year
        ;;i)  use a loop to enter buy orders
        
        (for [n (range 20)]  ; buy
            (order roe_sorted[n+1] 1)
        )
    
        
        ;;3. update the trading records (eval-report)
        (update-eval-report (get-date))
        
        ; move on to the next trading day
        (next-date)
        
        ;;4. if equals end-date, sell all stocks
        (for [n (range 20)]  ; sell
            (order roe_sorted[n+1] -1)
        )
    )
)

Syntax error macroexpanding clojure.core/let at (REPL:27:13).
roe-20 - failed: vector? at: [:bindings] spec: :clojure.core.specs.alpha/bindings


class clojure.lang.Compiler$CompilerException: 

### Check order record

In [5]:
(pprint/print-table (deref order-record))


|      :date | :tic |  :price | :quantity | :reference |
|------------+------+---------+-----------+------------|
| 1980-12-17 | AAPL | 25.9375 |        50 |          2 |
| 1980-12-18 | AAPL | 26.6875 |       -10 |          3 |
| 1980-12-22 | AAPL | 29.6875 |       -10 |          5 |
| 1980-12-24 | AAPL | 32.5625 |       -10 |          7 |
| 1980-12-29 | AAPL | 36.0625 |       -10 |          9 |
| 1980-12-31 | AAPL | 34.1875 |       -10 |         11 |


nil

### Check portfolio record

In [6]:
;; view final portfolio
(view-portfolio)


| :asset |  :price | :aprc | :quantity | :tot-val |
|--------+---------+-------+-----------+----------|
|   cash |     N/A |   N/A |       N/A |    10295 |
|   AAPL | 34.1875 | 29.42 |         0 |        0 |


nil

In [7]:
;; view portfolio value and return
(view-portfolio-record)


|      :date | :tot-value | :daily-ret | :loan | :leverage |
|------------+------------+------------+-------+-----------|
| 1980-12-16 |     $10000 |      0.00% | $0.00 |     0.00% |
| 1980-12-17 |     $10007 |      0.00% | $0.00 |     0.00% |
| 1980-12-18 |     $10026 |      0.00% | $0.00 |     0.00% |
| 1980-12-22 |     $10096 |      0.01% | $0.00 |     0.00% |
| 1980-12-24 |     $10168 |      0.01% | $0.00 |     0.00% |
| 1980-12-29 |     $10254 |      0.01% | $0.00 |     0.00% |
| 1980-12-31 |     $10295 |      0.00% | $0.00 |     0.00% |


nil

### Generate evaluation report

In [8]:
(eval-report)


|      :date | :pnl-pt | :ret-da | :ret-r | :ret-tot | :sharpe-e | :sharpe-r | :tot-val | :vol-e | :vol-r |
|------------+---------+---------+--------+----------+-----------+-----------+----------+--------+--------|
| 1980-12-16 |      $7 |   0.08% |  0.00% |    0.08% |     1.41% |     0.22% |   $10007 |  0.06% |  0.01% |
| 1980-12-17 |     $13 |   0.19% |  0.01% |    0.27% |     2.81% |     0.45% |   $10026 |  0.10% |  0.02% |
| 1980-12-18 |     $13 |   0.19% |  0.01% |    0.27% |     2.81% |     0.45% |   $10026 |  0.10% |  0.02% |
| 1980-12-19 |     $32 |   0.70% |  0.00% |    0.96% |     3.07% |     0.10% |   $10096 |  0.31% |  0.05% |
| 1980-12-22 |     $32 |   0.70% |  0.00% |    0.96% |     3.07% |     0.10% |   $10096 |  0.31% |  0.05% |
| 1980-12-23 |     $42 |   0.71% |  0.01% |    1.67% |     4.88% |     0.11% |   $10168 |  0.34% |  0.05% |
| 1980-12-24 |     $42 |   0.71% |  0.01% |    1.67% |     4.88% |     0.11% |   $10168 |  0.34% |  0.05% |
| 1980-12-26 |     $50 |   

nil

### Plot variables
Below are example codes that show how to plot different variables in the portfolio record / evaluation record.

### 1. Portfolio daily return

In [9]:
(def data (deref portfolio-value))

#'clojure-backtesting.demo/data

In [10]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "portfolio")
  data))

#'clojure-backtesting.demo/data-to-plot

In [11]:
(first data-to-plot)

{:date "1980-12-16", :tot-value 10000, :daily-ret 0.0, :loan 0.0, :leverage 0.0, :plot "portfolio"}

In [12]:
(plot data-to-plot :plot :date :daily-ret)

### 2. Plot volatility (expanded window)

In [13]:
(def data (deref eval-record))

#'clojure-backtesting.demo/data

In [14]:
(first data)

{:date "1980-12-16", :pnl-pt 7.806415917975755, :ret-da 0.07803370496319298, :ret-r 0.0019683764070455023, :ret-tot 7.803370496319297E-4, :sharpe-e 1.4142135623730954, :sharpe-r 0.22471944370971253, :tot-val 10007.806415917976, :vol-e 0.05517816194058409, :vol-r 0.00875926165778608}

In [15]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "volatility")
  data))

#'clojure-backtesting.demo/data-to-plot

In [16]:
(eval-report)


|      :date | :pnl-pt | :ret-da | :ret-r | :ret-tot | :sharpe-e | :sharpe-r | :tot-val | :vol-e | :vol-r |
|------------+---------+---------+--------+----------+-----------+-----------+----------+--------+--------|
| 1980-12-16 |      $7 |   0.08% |  0.00% |    0.08% |     1.41% |     0.22% |   $10007 |  0.06% |  0.01% |
| 1980-12-17 |     $13 |   0.19% |  0.01% |    0.27% |     2.81% |     0.45% |   $10026 |  0.10% |  0.02% |
| 1980-12-18 |     $13 |   0.19% |  0.01% |    0.27% |     2.81% |     0.45% |   $10026 |  0.10% |  0.02% |
| 1980-12-19 |     $32 |   0.70% |  0.00% |    0.96% |     3.07% |     0.10% |   $10096 |  0.31% |  0.05% |
| 1980-12-22 |     $32 |   0.70% |  0.00% |    0.96% |     3.07% |     0.10% |   $10096 |  0.31% |  0.05% |
| 1980-12-23 |     $42 |   0.71% |  0.01% |    1.67% |     4.88% |     0.11% |   $10168 |  0.34% |  0.05% |
| 1980-12-24 |     $42 |   0.71% |  0.01% |    1.67% |     4.88% |     0.11% |   $10168 |  0.34% |  0.05% |
| 1980-12-26 |     $50 |   

nil

In [17]:
;(println data-to-plot)

In [6]:
(plot data-to-plot :plot :date :vol-e :pnl-pt)

Syntax error compiling at (REPL:1:1).
Unable to resolve symbol: data-to-plot in this context


class clojure.lang.Compiler$CompilerException: 

### 3. Plot sharpe ratio (expanded window)

In [19]:
; Add rename legend
(def data-to-plot
 (map #(assoc % :plot "sharpe ratio")
  data))

(plot data-to-plot :plot :date :sharpe-e)

### 4. Plot stock price

In [20]:
(def data (deref order-record))

#'clojure-backtesting.demo/data

In [21]:
(first data)

{:date "1980-12-17", :tic "AAPL", :price 25.9375, :quantity 50, :reference 2}

In [22]:
; Add rename legend
(def data-to-plot
 (map #(assoc % :plot "price")
  data))

(plot data-to-plot :plot :date :price)